# **「PyTorch入門 8. クイックスタート」**

## **データの取り扱い**(数据的处理)

用PyTorch获取数据时,有两个基本要素

``torch.utils.data.Dataset``和``torch.utils.data.DataLoader``

``Dataset`` 中包括每个与标签对应的样本数据

``DataLoader``对``Dataset``进行封装,以便对其进行迭代操作

In [ ]:
%matplotlib inline

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

PyTorch中包含几个固有的libarary

- TorchText
- TorchVision
- TorchAudio

``torchvision.datasets`` 模块,图像数据提供了大量Dataset对象.

TorchVision中所有的Dataset都有两个参数

``transform`` 和 ``target_transform`` 分别对样本和标签进行转换处理

In [ ]:
# 下载训练数据集
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 下载测试数据集
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:00<00:00, 116556440.05it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 4668686.17it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




100%|██████████| 4422102/4422102 [00:00<00:00, 60029776.31it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 7012756.41it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



Dataset作为参数传入DataLoader

DataLoader 对Dataset进行封装,以便对其进行迭代。

In [ ]:
# batch设定为 64
# 这意味着DataLoader会返回由64个特征和标签组成的batch
batch_size = 64

# 创建DataLoader
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


## **モデルの構築**(model的构建)

PyTorch中定义神经网络时,需继承``nn.Moudule``

``__init__``函数用于定义网络中的各层,数据的正向传播用``forward``函数定义

并且想要高速运行,可以在GPU上运行

In [ ]:
# 如果可以在GPU上运行时,在GPU上运行
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# 定义model
class NeuralNetwork(nn.Module):
    def __init__(self):

        # 调用父类的构造函数，确保父类nn.Module的初始化也被执行
        super(NeuralNetwork, self).__init__()

        # 定义了一个扁平化层，将输入的28x28图像转换为784维的一维向量
        # 第0维度代表样本编号，该维度不会通过nn.Flatten发生变化
        self.flatten = nn.Flatten()

        # 定义了一系列线性层和ReLU激活函数组成的顺序容器（sequential container）
        self.linear_relu_stack = nn.Sequential(

            # 第一个线性层，将输入的784维向量映射到512维
            nn.Linear(28*28, 512),
            # 一个ReLU激活函数，应用非线性变换
            nn.ReLU(),

            # 第二个线性层，将512维向量映射到另一个512维向量
            nn.Linear(512, 512),
            # 再次应用nn.ReLU()
            nn.ReLU(),

            # 第三个线性层，将512维向量映射到10维向量
            nn.Linear(512, 10),
            # 再应用一次nn.ReLU()非线性变换。
            nn.ReLU()
        )

    def forward(self, x):

        # 扁平化输入
        x = self.flatten(x)

        # 将扁平化后的向量通过定义的线性和ReLU层的堆栈
        logits = self.linear_relu_stack(x)

        # 返回最终的输出
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [ ]:
# 将model移至GPU
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


## **モデルパラメータの最適化**(model的超参数最优化)

为了训练神经网络

loss function和optimizer是必要的

In [ ]:
# 声明loss function
loss_fn = nn.CrossEntropyLoss()

# 声明optimizer learn rate = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

在训练过程中,model首先对一批(batch)的数据进行推算,计算损失(loss)

然后反向传播损失并更新模型参数



In [ ]:
# 定义训练函数
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    # 迭代遍历数据
    # X是输入数据,y是标签
    for batch, (X, y) in enumerate(dataloader):

        # 将数据装载到device上
        X, y = X.to(device), y.to(device)

        # 预测值与损失的计算
        pred = model(X)
        loss = loss_fn(pred, y)

        # 向后传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 每进行100批次,打印一次当前损失和训练进度
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

训练后的model,通过test dataset对性能进行评价

In [ ]:
def test(dataloader, model):
    size = len(dataloader.dataset)

    # 将model变为评价模式
    model.eval()

    # 初始化test_loss,correct
    test_loss, correct = 0, 0

    # 禁止梯度计算
    with torch.no_grad():

        for X, y in dataloader:
            # 将数据装载到device上
            X, y = X.to(device), y.to(device)
            pred = model(X)

            # 累加损失率
            test_loss += loss_fn(pred, y).item()
            # 累加正确率
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    # 计算平均损失率
    test_loss /= size
    # 计算平均正确率
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
# 迭代的次数(纪元数)
epochs = 20

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    # 训练
    train(train_dataloader, model, loss_fn, optimizer)
    # 测试
    test(test_dataloader, model)

print("Done!")

Epoch 1
-------------------------------
loss: 2.251115  [    0/60000]
loss: 2.264123  [ 6400/60000]
loss: 2.260548  [12800/60000]
loss: 2.256598  [19200/60000]
loss: 2.235826  [25600/60000]
loss: 2.195851  [32000/60000]
loss: 2.208276  [38400/60000]
loss: 2.187165  [44800/60000]
loss: 2.184657  [51200/60000]
loss: 2.142739  [57600/60000]
Test Error: 
 Accuracy: 34.4%, Avg loss: 0.034216 

Epoch 2
-------------------------------
loss: 2.196593  [    0/60000]
loss: 2.207914  [ 6400/60000]
loss: 2.203788  [12800/60000]
loss: 2.190911  [19200/60000]
loss: 2.166761  [25600/60000]
loss: 2.110148  [32000/60000]
loss: 2.120334  [38400/60000]
loss: 2.102041  [44800/60000]
loss: 2.098146  [51200/60000]
loss: 2.023584  [57600/60000]
Test Error: 
 Accuracy: 39.1%, Avg loss: 0.032782 

Epoch 3
-------------------------------
loss: 2.127317  [    0/60000]
loss: 2.133466  [ 6400/60000]
loss: 2.131960  [12800/60000]
loss: 2.106367  [19200/60000]
loss: 2.087126  [25600/60000]
loss: 2.014261  [32000/600

## **モデルの保存**(模型保存)

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## **モデルの読み込み**(读取model)

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))